# Chapter 6: Deep Q-Networks

Initially, improving on Q-iteration by being cleverer about which states to focus on, and how to update values. Then combining these techniques with neural nets for a whole new level of power...

## Q-learning

In [1]:
import gym
import collections
from torch.utils.tensorboard import SummaryWriter

ENV_NAME = "FrozenLake-v1"
GAMMA = 0.9
ALPHA = 0.2 # learning rate - "blending" parameter in the value update step later
TEST_EPISODES = 20

class Agent:
    def __init__(self):
        self.env = gym.make(ENV_NAME)
        self.state = self.env.reset()
        self.values = collections.defaultdict(float) # only storing values, not rewards or transitions any more!

    # Obtain next transition from environment
    def sample_env(self):
        action = self.env.action_space.sample()
        old_state = self.state
        new_state, reward, is_done, _ = self.env.step(action)
        self.state = self.env.reset() if is_done else new_state
        return old_state, action, reward, new_state

    # Given state, find "best" action according to our tabulated values
    # If we don't have a value for a state-action pair, assume it's 0
    def best_value_and_action(self, state):
        best_value, best_action = None, None
        for action in range(self.env.action_space.n):
            action_value = self.values[(state, action)]
            if best_value is None or best_value < action_value:
                best_value, best_action = action_value, action
        return best_value, best_action
    
    # Update values table using one step from environment
    def value_update(self, s, a, r, next_s):
        best_v, _ = self.best_value_and_action(next_s)
        new_v = r + GAMMA*best_v # Bellman approximation
        old_v = self.values[(s, a)]
        self.values[(s, a)] = (1-ALPHA)*old_v + ALPHA*new_v # blend old and new values
    
    # Play one full episode using provided test environment
    def play_episode(self, env):
        total_reward = 0.0
        state = env.reset()
        while True:
            _, action = self.best_value_and_action(state)
            new_state, reward, is_done, _ = env.step(action)
            total_reward += reward
            if is_done:
                break
            state = new_state
        return total_reward


In [2]:
test_env = gym.make(ENV_NAME)
agent = Agent()
writer = SummaryWriter(comment="-q-learning")

iter_no = 0
best_reward = 0.0
while True:
    iter_no += 1
    s, a, r, next_s = agent.sample_env()
    agent.value_update(s, a, r, next_s)

    reward = 0.0
    for _ in range(TEST_EPISODES):
        reward += agent.play_episode(test_env)
    reward /= TEST_EPISODES
    writer.add_scalar("reward", reward, iter_no)
    if reward > best_reward:
        print(f"Best reward updated {best_reward:.3f} -> {reward:.3f}")
        best_reward = reward
    if reward > 0.80:
        print(f"Solved in {iter_no} iterations!")
        writer.close()
        break


Best reward updated 0.000 -> 0.200
Best reward updated 0.200 -> 0.250
Best reward updated 0.250 -> 0.300
Best reward updated 0.300 -> 0.350
Best reward updated 0.350 -> 0.550
Best reward updated 0.550 -> 0.700
Best reward updated 0.700 -> 0.800
Best reward updated 0.800 -> 0.900
Solved in 5092 iterations!


## A more challenging problem: Atari games

In [8]:
# Wrappers

import cv2
import gym
import gym.spaces
import numpy as np
import collections

# Press FIRE button in games where that triggers game start
class FireResetEnv(gym.Wrapper):
    def __init__(self, env=None):
        super(FireResetEnv, self).__init__(env)
        # Check at least 3 actions, and that the second action is the "FIRE" button
        assert env.unwrapped.get_action_meanings()[1] == "FIRE"
        assert len(env.unwrapped.get_action_meanings()) >= 3

    def step(self, action):
        return self.env.step(action)

    # Also check a couple of corner cases for certain games, relating to actions 1 and 2
    def reset(self):
        self.env.reset()
        obs, _, done, _ = self.env.step(1)
        if done:
            self.env.reset()
        obs, _, done, _ = self.env.step(2)
        if done:
            self.env.reset()
        return obs


# Combine repetition of actions during K frames, and pixels from two consecutive frames
class MaxAndSkipEnv(gym.Wrapper):
    def __init__(self, env=None, skip=4):
        super(MaxAndSkipEnv, self).__init__(env)
        self._obs_buffer = collections.deque(maxlen=2)
        self._skip = skip # 4 is arbitrary, but canonically works well for Atari games

    def step(self, action):
        total_reward = 0.0
        done = None
        # Perform the same action _skip times (as in, skip the decision-making process)
        for _ in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            self._obs_buffer.append(obs)
            total_reward += reward
            if done:
                break
        max_frame = np.max(np.stack(self._obs_buffer), axis=0) # get brightest pixel from two frames
        return max_frame, total_reward, done, info
    
    def reset(self):
        self._obs_buffer.clear()
        obs = self.env.reset()
        self._obs_buffer.append(obs)
        return obs


# Rescale frames and convert to greyscale
class ProcessFrame84(gym.ObservationWrapper):
    def __init__(self, env=None):
        super(ProcessFrame84, self).__init__(env)
        self.observation_space = gym.spaces.Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8) # pixel intensities
    
    def observation(self, obs):
        return ProcessFrame84.process(obs)

    def process(frame):
        if frame.size == 210*160*3:
            img = np.reshape(frame, [210, 160, 3]).astype(np.float32)
        elif frame.size == 250*160*3:
            img = np.reshape(frame, [250, 160, 3]).astype(np.float32)
        else:
            assert False, "Unknown resolution" # throw an error
        img = 0.299*img[:, :, 0] + 0.587*img[:, :, 1] + 0.114*img[:, :, 2] # "colorimetric" greyscale averaging
        resized_screen = cv2.resize(img, (84, 110), interpolation=cv2.INTER_AREA)
        x_t = resized_screen[18:102, :] # crop top and bottom
        x_t = np.reshape(x_t, [84, 84, 1]) # add third dimension
        return x_t.astype(np.uint8)


# Create a stack of consecutive frames and return as a single observation
# (this allows the network to "see" velocity, movement etc which are not apparent from an isolated frame)
class BufferWrapper(gym.ObservationWrapper):
    def __init__(self, env, n_steps, dtype=np.float32):
        super(BufferWrapper, self).__init__(env)
        self.dtype = dtype
        old_space = env.observation_space
        self.observation_space = gym.spaces.Box(
            old_space.low.repeat(n_steps, axis=0),
            old_space.high.repeat(n_steps, axis=0),
            dtype=dtype
        )

    def reset(self):
        self.buffer = np.zeros_like(self.observation_space.low, dtype=self.dtype)
        return self.observation(self.env.reset())
    
    def observation(self, observation):
        self.buffer[:-1] = self.buffer[1:]
        self.buffer[-1] = observation
        return self.buffer


# Rearrange dimensions to PyTorch convention (channel-first)
class ImageToPyTorch(gym.ObservationWrapper):
    def __init__(self, env):
        super(ImageToPyTorch, self).__init__(env)
        old_shape = self.observation_space.shape
        new_shape = (old_shape[-1], old_shape[0], old_shape[1])
        self.observation_space = gym.spaces.Box(low=0.0, high=1.0, shape=new_shape, dtype=np.float32)

    def observation(self, observation):
        return np.moveaxis(observation, 2, 0)


# Convert observation data from bytes to float and rescale to 0-1
class ScaledFloatFrame(gym.ObservationWrapper):
    def observation(self, obs):
        return np.array(obs).astype(np.float32) / 255.0


# Function to create environment and apply all wrappers
def make_env(env_name):
    env = gym.make(env_name)
    env = MaxAndSkipEnv(env)
    env = FireResetEnv(env)
    env = ProcessFrame84(env)
    env = ImageToPyTorch(env)
    env = BufferWrapper(env, 4) # 4 is arbitrary, but canonically works well for Atari games
    env = ScaledFloatFrame(env)
    return env

In [11]:
# The model

import torch
import torch.nn as nn
import numpy as np

class DQN(nn.Module):
    def __init__(self, input_shape, n_actions):
        super(DQN, self).__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU()
        )

        # No built-in "flatten" layer in torch, so we have to DIY that bit and then use
        # a separate network for our final couple of fully-connected layers  
        conv_out_size = self._get_conv_out(input_shape)
        self.fc = nn.Sequential(
            nn.Linear(conv_out_size, 512),
            nn.ReLU(),
            nn.Linear(512, n_actions)
        )

    def _get_conv_out(self, shape):
        o = self.conv(torch.zeros(1, *shape))
        return int(np.prod(o.size()))

    def forward(self, x):
        conv_out = self.conv(x).view(x.size()[0], -1) # reshape to just batch number in 0th dimension, then all params in 1st
        return self.fc(conv_out)

In [15]:
# Training

import time
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

ENV_NAME = "PongNoFrameskip-v4"
DEVICE = "cpu"
MEAN_REWARD_BOUND = 19.0 # stop training after mean reward over last 100 episodes is >= this
GAMMA = 0.99 # discount factor for Bellman approximation
BATCH_SIZE = 32 # sampled from replay buffer
REPLAY_SIZE = 10000 # max capacity of buffer
REPLAY_START_SIZE = 10000 # frames before starting to populate buffer
LEARNING_RATE = 1e-4 # for use in Adam optimiser
SYNC_TARGET_FRAMES = 1000 # how frequently to sync model weights from training model to target model
EPSILON_DECAY_LAST_FRAME = 150000 # linearly decay epsilon over the first frames
EPSILON_START = 1.0
EPSILON_FINAL = 0.01


Experience = collections.namedtuple("Experience", field_names=["state", "action", "reward", "done", "new_state"])

class ExperienceBuffer:
    def __init__(self, capacity):
        self.buffer = collections.deque(maxlen=capacity)
    
    def __len__(self):
        return len(self.buffer)

    def append(self, experience):
        self.buffer.append(experience)

    def sample(self, batch_size):
        indices = np.random.choice(len(self.buffer), batch_size, replace=False)
        states, actions, rewards, dones, next_states = zip(*[self.buffer[idx] for idx in indices])
        # Repack into arrays for easier loss calculation shortly
        return np.array(states), np.array(actions), np.array(rewards, dtype=np.float32), np.array(dones, dtype=np.uint8), np.array(next_states)


class Agent:
    def __init__(self, env, exp_buffer):
        self.env = env
        self.exp_buffer = exp_buffer
        self._reset()

    def _reset(self):
        self.state = env.reset()
        self.total_reward = 0.0
    

    @torch.no_grad() # reduces memory when we're sure we won't need Tensor.backward(); sets default of requires_grad=False
    def play_step(self, net, epsilon=0.0, device="cpu"):
        done_reward = None

        # Random action taken with probability epsilon
        if np.random.random() < epsilon:
            action = env.action_space.sample()
        # Otherwise choose action using output from our neural net
        else:
            state_a = np.array([self.state], copy=False)
            state_v = torch.tensor(state_a).to(device)
            q_vals_v = net(state_v)
            _, act_v = torch.max(q_vals_v, dim=1)
            action = int(act_v.item())

        new_state, reward, is_done, _ = self.env.step(action)
        self.total_reward += reward

        exp = Experience(self.state, action, reward, is_done, new_state)
        self.exp_buffer.append(exp) # append new experience to buffer
        self.state = new_state
        if is_done:
            done_reward = self.total_reward
            self._reset()
        return done_reward # still None if we're not done


def calc_loss(batch, net, tgt_net, device="cpu"):
    states, actions, rewards, dones, next_states = batch # tuple of arrays, as packed by ExperienceBuffer.sample()

    # Wrap input in tensors and send to specified device (i.e. GPU, if specified)
    states_v = torch.tensor(np.array(states, copy=False)).to(device)
    next_states_v = torch.tensor(np.array(next_states, copy=False)).to(device)
    actions_v = torch.LongTensor(actions).to(device) # .gather needs int64 data later
    rewards_v = torch.tensor(rewards).to(device)
    done_mask = torch.BoolTensor(dones).to(device)

    # Get output from model, then keep only selected action for each batch
    # .unsqueeze() adds batch index needed by .gather(), .squeeze() drops extra resulting dimension
    state_action_values = net(states_v).gather(1, actions_v.unsqueeze(-1)).squeeze(-1)

    # Using target network, get max Q-value across dimension 1 (actions)
    # .max() returns values and indices, but we only want to keep values
    next_state_values = tgt_net(next_states_v).max(1)[0]
    next_state_values[done_mask] = 0.0 # training won't converge unless we stop rewarding future states once done!
    next_state_values = next_state_values.detach() # prevent gradient calculations from affecting tgt_net by detaching from computation graph

    expected_state_action_values = rewards_v + GAMMA*next_state_values # Bellman approximation
    return nn.MSELoss()(state_action_values, expected_state_action_values)


# Training loop
env = make_env(ENV_NAME)
net = DQN(env.observation_space.shape, env.action_space.n).to(DEVICE)
tgt_net = DQN(env.observation_space.shape, env.action_space.n).to(DEVICE)

writer = SummaryWriter(comment="-"+ENV_NAME)
print(net)

buffer = ExperienceBuffer(REPLAY_SIZE)
agent = Agent(env, buffer)
epsilon = EPSILON_START
optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)
total_rewards = []
frame_idx = 0
ts_frame = 0
ts = time.time()
best_m_reward = None

while True:
    frame_idx += 1
    epsilon = max(EPSILON_FINAL, EPSILON_START - frame_idx/EPSILON_DECAY_LAST_FRAME)

    reward = agent.play_step(net, epsilon, device=DEVICE)
    if reward is not None:
        total_rewards.append(reward)
        speed = (frame_idx - ts_frame) / (time.time() - ts)
        ts_frame = frame_idx
        ts = time.time()
        m_reward = np.mean(total_rewards[-100:])
        print(
            "%d: done %d games, reward %.3f, eps %.2f, speed %.2f f/s" %
                (frame_idx, len(total_rewards), m_reward, epsilon, speed)
        )
        writer.add_scalar("epsilon", epsilon, frame_idx)
        writer.add_scalar("speed", speed, frame_idx)
        writer.add_scalar("reward_100", m_reward, frame_idx)
        writer.add_scalar("reward", reward, frame_idx)

        if best_m_reward is None or best_m_reward < m_reward:
            torch.save(net.state_dict(), f"{ENV_NAME}-best_{m_reward:.0f}.dat")
            if best_m_reward is not None:
                print(f"Best reward updated {best_m_reward:.3f} -> {m_reward:.3f}")
            best_m_reward = m_reward

        if m_reward > MEAN_REWARD_BOUND:
            print(f"Solved in {frame_idx} frames!")
            break

    if len(buffer) < REPLAY_START_SIZE:
        continue

    if frame_idx % SYNC_TARGET_FRAMES == 0:
        tgt_net.load_state_dict(net.state_dict()) # Update target net weights every SYNC_TARGET_FRAMES frames

    optimizer.zero_grad()
    batch = buffer.sample(BATCH_SIZE)
    loss_t = calc_loss(batch, net, tgt_net, device=DEVICE)
    loss_t.backward()
    optimizer.step()


DQN(
  (conv): Sequential(
    (0): Conv2d(4, 32, kernel_size=(8, 8), stride=(4, 4))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
    (3): ReLU()
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (5): ReLU()
  )
  (fc): Sequential(
    (0): Linear(in_features=3136, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=6, bias=True)
  )
)
978: done 1 games, reward -20.000, eps 0.99, speed 211.44 f/s
2051: done 2 games, reward -19.500, eps 0.99, speed 363.04 f/s
Best reward updated -20.000 -> -19.500
2890: done 3 games, reward -20.000, eps 0.98, speed 398.66 f/s
3995: done 4 games, reward -19.500, eps 0.97, speed 435.54 f/s
4804: done 5 games, reward -19.800, eps 0.97, speed 396.37 f/s
6048: done 6 games, reward -19.333, eps 0.96, speed 426.32 f/s
Best reward updated -19.500 -> -19.333
6967: done 7 games, reward -19.429, eps 0.95, speed 369.17 f/s
7957: done 8 games, reward -19.625, eps 0.95, speed 342.94 f/s

KeyboardInterrupt: 

Sadly this will take FOREVER on my little CPU (the book estimates about a day and a half at best) so I guess I'll come back to this one in the magical future times when everyone has a GPU.